# Treinamento do modelo usando Random Forest no SageMaker

### Importando bibliotecas e variáveis de ambiente 

In [ ]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Carregar variáveis de ambiente a partir de um arquivo .env
load_dotenv()

# Definir a região a ser usada (us-east-1)
region_name = 'us-east-1'

# Carregar variáveis de ambiente para conectar ao banco de dados RDS
database_name = os.getenv('DATABASE_NAME')
table_name = os.getenv('TABLE_NAME')
rds_host = os.getenv('RDS_HOST')
rds_port = os.getenv('RDS_PORT')
rds_user = os.getenv('RDS_USER')
rds_password = os.getenv('RDS_PASSWORD')

# Criar uma sessão boto3
boto3_session = boto3.Session()

# Criar um cliente S3 usando a sessão boto3
s3_client = boto3_session.client('s3')

# Criar um cliente SageMaker usando a sessão boto3
sm_boto3 = boto3_session.client('sagemaker')

# Criar uma sessão SageMaker
sess = sagemaker.Session(boto_session=boto3_session)

# Obter a região da sessão boto3
region = sess.boto_session.region_name

# Definir o nome do bucket S3 a partir da variável de ambiente
bucket = os.getenv('BUCKET_NAME')

# Imprimir o nome do bucket que está sendo usado
print("Usando o bucket " + bucket)


### Importando a database do RDS

In [ ]:
# Criar a string de conexão para o SQLAlchemy
db_url = f'mysql+pymysql://{rds_user}:{rds_password}@{rds_host}:{rds_port}/{database_name}'

# Conectar-se ao banco de dados
engine = create_engine(db_url)

# Consultar o banco de dados e carregar os dados em um DataFrame
query = f'SELECT * FROM {table_name}'
hotel_reservas = pd.read_sql(query, con=engine)
hotel_reservas

### Tratamento de dados

In [ ]:
# Remover as colunas que não serão utilizadas na análise ou no modelo
hotel_reservas.drop(['Booking_ID','booking_status', 'no_of_previous_bookings_not_canceled','repeated_guest', 'no_of_previous_cancellations'], axis=1, inplace=True)

# Converter as colunas categóricas em variáveis dummy (one-hot encoding)
hotel_reservas = pd.get_dummies(hotel_reservas, prefix=['type_of_meal_plan', 'room_type_reserved', 'market_segment_type'],
                                columns=['type_of_meal_plan', 'room_type_reserved', 'market_segment_type'])

# Exibir o DataFrame transformado
hotel_reservas

In [ ]:
# Obter a lista de todas as colunas do DataFrame hotel_reservas
features = list(hotel_reservas.columns)

# Exibir a lista de features (nomes das colunas)
features

In [ ]:
# Remover e obter o décimo primeiro elemento da lista de features (coluna que será usada como label)
label = features.pop(10)

# Exibir o nome da coluna que foi removida da lista de features e armazenada em label
label

In [ ]:
# Selecionar as colunas restantes como variáveis independentes (features)
x = hotel_reservas[features]

# Selecionar a coluna removida anteriormente como variável dependente (label)
y = hotel_reservas[label]

In [ ]:
# Exibir as primeiras cinco linhas do DataFrame x (variáveis independentes)
x.head()

In [ ]:
# Exibir as primeiras cinco linhas do DataFrame y (variáveis independentes)
y.head()

In [ ]:
# Exibir a forma (número de linhas e colunas) do DataFrame x (variáveis independentes)
x.shape

In [ ]:
# Exibir a contagem de valores únicos na série y (variável dependente)
y.value_counts()

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [ ]:
# Exibir a forma (número de linhas e colunas) do conjunto de treinamento das variáveis independentes
print(X_train.shape)

# Exibir a forma (número de linhas e colunas) do conjunto de teste das variáveis independentes
print(X_test.shape)

# Exibir a forma (número de linhas) do conjunto de treinamento da variável dependente
print(y_train.shape)

# Exibir a forma (número de linhas) do conjunto de teste da variável dependente
print(y_test.shape)

In [ ]:
# Converter o conjunto de treinamento das variáveis independentes para um DataFrame
trainX = pd.DataFrame(X_train)

# Adicionar a variável dependente ao conjunto de treinamento
trainX[label] = y_train

# Converter o conjunto de teste das variáveis independentes para um DataFrame
testX = pd.DataFrame(X_test)

# Adicionar a variável dependente ao conjunto de teste
testX[label] = y_test

In [ ]:
# Exibir a forma (número de linhas e colunas) do DataFrame de treinamento completo (variáveis independentes + dependente)
print(trainX.shape)

# Exibir a forma (número de linhas e colunas) do DataFrame de teste completo (variáveis independentes + dependente)
print(testX.shape)